In [1]:
import Config
import ann_helper

import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

import multiprocessing
import datetime
import time
import copy

from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay, precision_score
import tensorflow as tf
from tensorflow.keras import regularizers
from keras.layers import Dropout
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from pymongo import MongoClient
client=MongoClient(Config.DB_Hostname,Config.DB_Port)

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def clustering():
    
    return None
    #setting up pnl matrix
    self.predicted_params
    pnl_matrix = pd.DataFrame()
    for strategy_variant in range(1, self.predicted_params.strategy_variant.max() + 1):
        pnl_matrix[strategy_variant] = self.predicted_params[self.predicted_params.strategy_variant == strategy_variant].sort_index().sort_index().second_drawdown_pnl.tolist()[:-1]

    pnl_matrix["date"]= sorted(self.predicted_params.index.unique().tolist()[:-1])
    pnl_matrix.set_index('date', inplace = True)
    pnl_matrix /= 100000
    scaler = StandardScaler()
    pnl_matrix = pd.DataFrame(scaler.fit_transform(pnl_matrix), 
                 columns = list(range(1, self.predicted_params.strategy_variant.max() + 1)),
                index = pnl_matrix.index)

    #get profitability
    predictions  = pd.DataFrame(client.Strategy.Ensemble_Prediction.find()).drop(columns = ["_id"]).set_index('date') # set to class variable
    XGB=[]
    ANN=[]

    for strat_variant in predictions.strategy_variant.unique():
        temp = predictions[predictions.strategy_variant==strat_variant]
        temp = pd.merge(temp, self.predicted_params[self.predicted_params.strategy_variant==strat_variant], left_index=True, right_index=True)
        XGB.append(precision_score(temp.Profitability.tolist(), temp.XGB_prediction.tolist()))
        ANN.append(precision_score(temp.Profitability.tolist(), temp.ANN_prediction.tolist()))

    results = {}

    #performing PCA
    pnl_matrix_pca = PCA(n_components = 64)
    pnl_matrix_pca.fit(pnl_matrix) 

    #setup coordinates
    distance_matrix = pd.DataFrame()
    for i in range(len(pnl_matrix_pca.components_)):
        distance_matrix[i]=pnl_matrix_pca.components_[i]
    distance_matrix.index = pnl_matrix.columns

    ##Add Profitability
    distance_matrix["XGB"] = XGB
    distance_matrix["ANN"] = ANN
    #distance_matrix = distance_matrix[["XGB", 'ANN']]

    #normalize distance matrix
    X = normalize(distance_matrix.to_numpy())

    #searching for clusters
    K = range(2, 100)
    fits = []
    score = []
    inertia = []
    for k in K:
        # train the model for current value of k on training data
        model = KMeans(n_clusters = k, random_state = 0, n_init='auto').fit(X)

        # append the model to fits
        fits.append(model)

        # Append the silhouette score to scores
        score.append(silhouette_score(X, model.labels_, metric='euclidean'))
        inertia.append(model.inertia_)

    cluster_sil =  np.array(score).argmin()+1
    score_sil = score[cluster_sil - 1]
    cluster_inertia =  np.array(inertia).argmin()+1
    score_inertia = score[cluster_inertia - 1]

    model = KMeans(n_clusters = 34, random_state = 0, n_init='auto').fit(X)
    cluster_map = []
    for variant, cluster in enumerate(model.labels_):
        cluster_map.append({"strategy_variant" : variant + 1, "cluster" : cluster + 1})
    cluster_map = pd.DataFrame(cluster_map)
    cluster_map["underlying"] = self.underlying

    print(client[f'{Config.Data_DB}']['Strategy_Clusters'].delete_many({"underlying":self.underlying}).deleted_count, f"records deleted for {self.underlying}")
    print(len(client[f'{Config.Data_DB}']['Strategy_Clusters'].insert_many(cluster_map.to_dict('records')).inserted_ids), f"records inserted for {self.underlying}")


class Cluster_Classifier:
    
    def __init__(self, start_date, end_date, prediction_date, underlying):
        
        self.start_date=start_date #2017-01-01
        self.end_date=end_date #2017-01-01
        self.prediction_date=prediction_date #2017-01-01
        self.underlying=underlying #NIFTY,BANKNIFTY
        self.variant_cluster_map = None
        self.XGBoost_Parameters = None
        self.current_week_expiry_today_map = None
        self.current_week_expiry_yesterday_map = None
        if self.start_date>=self.prediction_date:
            raise Exception("Start date has to be before Prediction Date")
        
        start=time.time()
        self.days_to_expiry = pd.DataFrame(client[f'{Config.Data_DB}']['Days_To_Expiry'].find({"underlying":self.underlying}))
        self.days_to_expiry.drop(columns = ['_id','underlying'],inplace=True)
        self.days_to_expiry.sort_values('date',inplace=True)
        self.days_to_expiry=self.days_to_expiry[(self.days_to_expiry.date>=self.start_date)&(self.days_to_expiry.date<=self.end_date)]
        self.days_to_expiry.set_index('date',inplace=True)
        self.days_to_expiry["current_week_yesterday"] = self.days_to_expiry.current_week.shift()
        self.days_to_expiry.dropna(inplace= True)
        self.current_week_expiry_today_map = self.days_to_expiry.current_week.to_dict()
        self.current_week_expiry_yesterday_map = self.days_to_expiry.current_week_yesterday.to_dict()
        self.days_to_expiry.drop(columns = ['current_week_yesterday'], inplace = True)
        print(f"Expiry Dates Downloaded in {time.time()-start} seconds")
        
        start=time.time()
        self.predicted_params=pd.DataFrame(client[f'{Config.Data_DB}'][f'{self.underlying}_Predicted_Parameters'].find())
        self.predicted_params.drop(columns=['_id'],inplace=True)
        self.predicted_params.sort_values(['date','strategy_variant'],inplace=True)
        self.predicted_params=self.predicted_params[(self.predicted_params.date>=self.start_date)&(self.predicted_params.date<=self.end_date)]
        self.predicted_params.set_index('date',inplace=True)
        self.predicted_params["profitability"] = np.where(self.predicted_params.second_drawdown_pnl >= 0, 1, 0)
        self.predicted_params_raw = self.predicted_params[['strategy_variant','pos_param','mov_param','prof_param', 'second_drawdown_pnl', 'profitability']]
        self.predicted_params.drop(columns=['pos_param', 'mov_param', 'prof_param', 'PnL', 'Drawdown_Count', 'Trade_Count', 'first_drawdown_pnl'],inplace=True)
        print(f"Predicted Parameters Downloaded in {time.time() - start} seconds")
                
        start=time.time()
        self.strategy_clusters=pd.DataFrame(client[f'{Config.Data_DB}']['Strategy_Clusters'].find({"underlying" : self.underlying}))
        self.strategy_clusters = self.strategy_clusters.sort_values('strategy_variant').drop(columns=['_id'])
        self.variant_cluster_map = self.strategy_clusters.set_index('strategy_variant').cluster.to_dict()
        print(f"Strategy Clusters Downloaded in {time.time()-start} seconds")

        start=time.time()
        self.ensemble_predictions = pd.DataFrame(client[f'{Config.Data_DB}']['Ensemble_Prediction'].find({"underlying" : self.underlying}))
        self.ensemble_predictions = self.ensemble_predictions.sort_values(['date', 'strategy_variant']).drop(columns=['_id'])
        self.ensemble_predictions = self.ensemble_predictions[(self.ensemble_predictions.date >= self.start_date) & (self.ensemble_predictions.date <= self.end_date)]
        self._process_ensemble_predictions()
        print(f"Ensemble Predictions Downloaded in {time.time()-start} seconds")
    
    def _process_ensemble_predictions(self):
        
        ensemble_predictions = []
        for strategy_variant in range(1, self.ensemble_predictions.strategy_variant.max() + 1):

            temp = self.ensemble_predictions[self.ensemble_predictions.strategy_variant == strategy_variant].set_index('date').drop(columns = ['strategy_variant'])
            temp = pd.merge(temp, self.predicted_params[self.predicted_params.strategy_variant == strategy_variant], left_index=True, right_index=True)

            precision_score_XGB = {}
            precision_score_ANN = {}
            for i in range(100, len(temp)+1):
                precision_temp = temp[i-100:i]
                precision_score_XGB[precision_temp.index[-1]] = precision_score(precision_temp.profitability.tolist(), precision_temp.XGB_prediction.tolist())
                precision_score_ANN[precision_temp.index[-1]] = precision_score(precision_temp.profitability.tolist(), precision_temp.ANN_prediction.tolist())

            temp["precision_score_XGB"] = temp.index.map(precision_score_XGB)
            temp["precision_score_ANN"] = temp.index.map(precision_score_ANN)
            ensemble_predictions.append(temp.dropna().reset_index())
            
        self.ensemble_predictions = pd.concat(ensemble_predictions).sort_values(['date', 'strategy_variant'])
        self.ensemble_predictions["cluster"] = self.ensemble_predictions.strategy_variant.map(self.strategy_clusters.set_index('strategy_variant').cluster.to_dict())

    def _get_splitted_parameters(self, start_split_index, end_split_index, cluster = None, strategy_variant = None):
        
        
        date_list_train = sorted(self.ensemble_predictions.date.unique())[:-start_split_index]
        if end_split_index != 0:
            date_list_test = sorted(self.ensemble_predictions.date.unique())[-start_split_index: -end_split_index]
        else:
            date_list_test = sorted(self.ensemble_predictions.date.unique())[-start_split_index:]
            
        ensemble_predictions = self.ensemble_predictions[(self.ensemble_predictions.XGB_prediction == 1) | (self.ensemble_predictions.ANN_prediction == 1)]
        ensemble_predictions_train = ensemble_predictions[ensemble_predictions.date.isin(date_list_train)]
        
        if strategy_variant is None:
            
            ensemble_predictions_test = ensemble_predictions[ensemble_predictions.date.isin(date_list_test)]
            
            X_train = ensemble_predictions_train[ensemble_predictions_train.cluster == cluster][['XGB_prediction', 'ANN_prediction', 'precision_score_XGB', 'precision_score_ANN']].values
            X_test = ensemble_predictions_test[ensemble_predictions_test.cluster == cluster][['XGB_prediction', 'ANN_prediction', 'precision_score_XGB', 'precision_score_ANN']].values
            y_train = ensemble_predictions_train[ensemble_predictions_train.cluster == cluster][['profitability']].values
            y_test = ensemble_predictions_test[ensemble_predictions_test.cluster == cluster][['profitability']].values
        
        else:
            
            ensemble_predictions_test = self.ensemble_predictions[self.ensemble_predictions.date.isin(date_list_test)]
            
            X_train = ensemble_predictions_train[ensemble_predictions_train.cluster == self.variant_cluster_map[strategy_variant]][['XGB_prediction', 'ANN_prediction', 'precision_score_XGB', 'precision_score_ANN']].values
            X_test = ensemble_predictions_test[ensemble_predictions_test.strategy_variant == strategy_variant][['XGB_prediction', 'ANN_prediction', 'precision_score_XGB', 'precision_score_ANN']].values
            y_train = ensemble_predictions_train[ensemble_predictions_train.cluster == self.variant_cluster_map[strategy_variant]][['profitability']].values
            y_test = ensemble_predictions_test[ensemble_predictions_test.strategy_variant == strategy_variant][['profitability']].values

        return X_train, X_test, y_train, y_test

    def _XGB(self, params):

        try:

            model = XGBClassifier(nthread = -1, n_estimators = params['n_estimators'], learning_rate = params['learning_rate'], 
                                  max_depth = params['max_depth'], reg_lambda = params['reg_lambda'], subsample = params['subsample'])

            model.fit(params['X_train'], params['y_train'], 
                      sample_weight = [params['sample_weight'] if y==0 else 1 for y in params['y_train'].ravel()])

            return model.predict(params['X_test'])

        except Exception as e:
            print(e, ". Error in running XGBoost")
            return np.zeros(len(params['X_test'])).reshape(-1,1)

    def _eval_hyperparams(self, summary_, size):

        summary = summary_.copy()

        if summary[summary.precision==1].num_signals.max() >= 2 * size:
            summary = summary[summary.precision==1]

        elif len(summary[(summary.precision>=0.75)&(summary.precision<1)]) > 0:
            summary = summary[(summary.precision>=0.75)&(summary.precision<1)]

        elif len(summary[(summary.precision>=0.6)&(summary.precision<0.75)]) > 0:
            summary = summary[(summary.precision>=0.6)&(summary.precision<0.75)]

        elif len(summary[(summary.precision>=0.5)&(summary.precision<0.6)]) > 0:
            summary = summary[(summary.precision>=0.5)&(summary.precision<0.6)]

        elif len(summary[(summary.precision>=0.3)&(summary.precision<0.5)]) > 0:
            summary = summary[(summary.precision>=0.3)&(summary.precision<0.5)]

        summary_subset = summary[summary.precision >= summary.precision.quantile(0.75)]
        return summary_subset.sort_values(['True_Positive','precision'], ascending=False).head(1)

    def _XGB_Grid_Simulator(self, X_train, X_test, y_train, y_test):

        max_depth_array = [2, 3, 4, 5, 6, 7] #no change
        n_estimators_array = [10, 20, 30, 40, 50, 75, 100] #no change
        learning_rate_array = [0.01, 0.05, 0.075, 0.1, 0.25, 0.5, 1] #no change
        reg_lambda_array = [0, 0.01, 0.05, 0.1] #no change
        subsample_array = [0.9, 0.8, 0.7] #no change
        sample_weight_array= [1, 2, 3, 4, 5, 10, 25, 50, 75, 100, 200]

        counter = 0
        summary = pd.DataFrame(columns =['max_depth', 'n_estimators', 'learning_rate', 'reg_lambda',
                                 'subsample', 'sample_weight', 'precision', 'num_signals'])

        for max_depth in max_depth_array:
            for n_estimators in n_estimators_array:
                for learning_rate in learning_rate_array:
                    for reg_lambda in reg_lambda_array:
                        for subsample in subsample_array:
                            for sample_weight in sample_weight_array:

                                params = {'max_depth' : max_depth, 'n_estimators': n_estimators, 'learning_rate' : learning_rate,
                                         'reg_lambda' : reg_lambda, 'subsample' : subsample, 'sample_weight' : sample_weight,
                                         'X_train' : X_train, 'X_test' : X_test, 'y_train' : y_train}

                                y_pred = self._XGB(params)
                                summary.loc[counter] = [max_depth, n_estimators, learning_rate, reg_lambda, subsample, 
                                                        sample_weight, precision_score(y_test, y_pred), sum(y_pred)]

                                counter+=1

        return summary

    def Hyperparameter_Tuner(self, start_cluster = 0):

        splits = [301, 201, 101]
        cluster_size_map = self.strategy_clusters.groupby('cluster').count().strategy_variant.to_dict()

        for cluster in range(1, 1 + self.strategy_clusters.cluster.max()):

            if cluster < start_cluster:
                continue

            start = time.time()
            print(f"Tuning XGBoost for cluster {cluster}")
            precision_list_XGB = []
            num_signal_list_XGB = []

            for split in splits:

                #preparing datasets
                X_train_split, X_test_split, y_train_split, y_test_split = self._get_splitted_parameters(split, split-100, cluster = cluster)
                
                #XGBoost grid sim
                summary_XGB = self._XGB_Grid_Simulator(X_train_split, X_test_split, y_train_split, y_test_split)
                summary_XGB.num_signals *= 100/len(X_test_split)
                precision_list_XGB.append(summary_XGB.precision.tolist())
                num_signal_list_XGB.append(summary_XGB.num_signals.tolist())

            #best XGB params
            summary_XGB.precision = np.array(precision_list_XGB).mean(axis=0)
            summary_XGB.num_signals = np.array(num_signal_list_XGB).mean(axis=0)
            summary_XGB["True_Positive"] = summary_XGB.precision * summary_XGB.num_signals
            summary_XGB = self._eval_hyperparams(summary_XGB, cluster_size_map[cluster])
            summary_XGB["underlying"] = self.underlying
            summary_XGB["cluster"] = cluster
            summary_XGB["date"] = self.end_date
            
            #inserting into cluster_XGB DB
            print(client[f'{Config.Data_DB}']['XGBoost_Cluster_Params'].delete_many({"date" : self.end_date, 'underlying' : self.underlying,
            "cluster": cluster}).deleted_count, f" documents deleted for {self.underlying} and {self.end_date} from XGBoost_Cluster_Params")
            print(len(client[f'{Config.Data_DB}']['XGBoost_Cluster_Params'].insert_many(summary_XGB.to_dict('records'
            )).inserted_ids),f" documents entered for {self.underlying} and {self.end_date} into XGBoost_Cluster_Params")
            print(f"completed in {round(time.time()-start)} seconds\n")

    def _get_XGB_params(self):
        
        params = pd.DataFrame(client[f'{Config.Data_DB}']['XGBoost_Cluster_Params'].find({"underlying" : self.underlying}))
        params = params[params.date == sorted(params.date.unique())[-1]]
        params.drop(columns = ['_id', 'date', 'underlying', 'precision', 'num_signals', 'True_Positive'], inplace = True)
        params = params.sort_values('cluster').set_index('cluster')
        
        params.max_depth = params.max_depth.astype(int)
        params.n_estimators = params.n_estimators.astype(int)
        params.learning_rate = params.learning_rate.astype(float)
        params.reg_lambda = params.reg_lambda.astype(float)
        params.subsample = params.subsample.astype(float)
        params.sample_weight = params.sample_weight.astype(float)
        
        self.XGBoost_Parameters = params.to_dict('records')

    def Predictor(self, start_variant = 0):

        self._get_XGB_params()    
        date_list = self.ensemble_predictions.date.sort_values().unique()
        initial_location = np.where(date_list == self.prediction_date)[0][0]
        summary_list = []

        for strategy_variant in range(self.ensemble_predictions.strategy_variant.max()):

            if strategy_variant < start_variant-1:
                continue

            start = time.time()
            print(f"Starting XGB prediction for strategy_variant {strategy_variant+1}")
            initial_loc = initial_location
            summary_XGB = pd.DataFrame(columns = ['date', 'underlying', 'strategy_variant', 'prediction'])

            for date in sorted(date_list[date_list >= self.prediction_date]):

                X_train, X_test, y_train, y_test = self._get_splitted_parameters(len(date_list) - initial_loc, len(date_list) - initial_loc - 1, strategy_variant = strategy_variant + 1)
                if X_test[0][0] == 0 and X_test[0][1] == 0:
                    prediction = 0

                else:
                    #XGBoost Params
                    params_XGB = copy.deepcopy(self.XGBoost_Parameters[self.variant_cluster_map[strategy_variant + 1] - 1])
                    params_XGB['X_train'] = X_train
                    params_XGB['X_test'] = X_test
                    params_XGB['y_train'] = y_train
                    prediction = self._XGB(params_XGB).ravel()[0]

                summary_XGB.loc[initial_loc] = [date, self.underlying, strategy_variant + 1, prediction]
                initial_loc += 1

            #return summary_XGB
            print(client[f'{Config.Data_DB}']['Clustered_Predictions'].delete_many({"date":{"$in":summary_XGB.date.unique().tolist()}, 
            "strategy_variant": strategy_variant + 1, "underlying" : self.underlying}).deleted_count, f"records deleted for strategy_variant {strategy_variant+1}")
            print(len(client[f'{Config.Data_DB}']['Clustered_Predictions'].insert_many(summary_XGB.to_dict('records')).inserted_ids),
                  f"records inserted for {self.underlying} strategy_variant {strategy_variant+1}")
            print(f"completed in {round(time.time()-start)} seconds\n")

    def Live_Strategies(self):

        predictions = pd.DataFrame(client.Strategy.Clustered_Predictions.find({"underlying" : self.underlying})).drop(columns = ['_id'])
        final_params = []

        for strategy_variant in range(1, predictions.strategy_variant.max() + 1):

            temp = pd.merge(self.predicted_params_raw[self.predicted_params_raw.strategy_variant == strategy_variant].drop(columns = ['strategy_variant']),
                            predictions[predictions.strategy_variant == strategy_variant].set_index('date'), left_index = True, right_index = True)

            if (strategy_variant - 1) % 24 > 11:
                temp["expiry_flag"] = temp.index.map(self.current_week_expiry_yesterday_map)
            else:
                temp["expiry_flag"] = temp.index.map(self.current_week_expiry_today_map)

            temp.prediction = np.where(temp.expiry_flag == 1, 0, temp.prediction)
            temp["returns"] = (temp.second_drawdown_pnl * temp.prediction) / 100000
            final_params.append(temp.reset_index())

        final_params = pd.concat(final_params).sort_values(['date','strategy_variant'])
        final_params = final_params[['date', 'underlying', 'strategy_variant', 'pos_param', 'mov_param', 'prof_param', 'prediction', 'returns']]
        
        print(client[f'{Config.Data_DB}']['Live_Strategies'].delete_many({"underlying" : self.underlying}).deleted_count, f" records deleted for {self.underlying}")
        print(len(client[f'{Config.Data_DB}']['Live_Strategies'].insert_many(final_params.to_dict('records')).inserted_ids), f"records inserted for {self.underlying}")

In [2]:
start_date = '2015-01-12' #starting data point best left untouchedd
end_date = '2024-01-19'#'2022-01-03'#'2023-07-03' # Last day for which you want prediction - should be next trading day
prediction_date = '2024-01-19' # should be legitimate and after start date and before end date. Only dates >= are processed and pushed into DB
underlying = 'NIFTY' #NIFTY/BANKNIFTY
self = Cluster_Classifier(start_date, end_date, prediction_date, underlying)


Expiry Dates Downloaded in 0.04770207405090332 seconds
Predicted Parameters Downloaded in 7.385295867919922 seconds
Strategy Clusters Downloaded in 0.005632877349853516 seconds
Ensemble Predictions Downloaded in 160.18596386909485 seconds


In [3]:
start = time.time()
#self.Hyperparameter_Tuner(start_cluster=0)
print(time.time()-start)

1.4781951904296875e-05


In [4]:
start = time.time()
self.Predictor()
print(time.time()-start)

Starting XGB prediction for strategy_variant 1
0 records deleted for strategy_variant 1
1 records inserted for NIFTY strategy_variant 1
completed in 0 seconds

Starting XGB prediction for strategy_variant 2
0 records deleted for strategy_variant 2
1 records inserted for NIFTY strategy_variant 2
completed in 0 seconds

Starting XGB prediction for strategy_variant 3
0 records deleted for strategy_variant 3
1 records inserted for NIFTY strategy_variant 3
completed in 0 seconds

Starting XGB prediction for strategy_variant 4
0 records deleted for strategy_variant 4
1 records inserted for NIFTY strategy_variant 4
completed in 0 seconds

Starting XGB prediction for strategy_variant 5
0 records deleted for strategy_variant 5
1 records inserted for NIFTY strategy_variant 5
completed in 0 seconds

Starting XGB prediction for strategy_variant 6
0 records deleted for strategy_variant 6
1 records inserted for NIFTY strategy_variant 6
completed in 0 seconds

Starting XGB prediction for strategy_var

0 records deleted for strategy_variant 54
1 records inserted for NIFTY strategy_variant 54
completed in 0 seconds

Starting XGB prediction for strategy_variant 55
0 records deleted for strategy_variant 55
1 records inserted for NIFTY strategy_variant 55
completed in 0 seconds

Starting XGB prediction for strategy_variant 56
0 records deleted for strategy_variant 56
1 records inserted for NIFTY strategy_variant 56
completed in 0 seconds

Starting XGB prediction for strategy_variant 57
0 records deleted for strategy_variant 57
1 records inserted for NIFTY strategy_variant 57
completed in 0 seconds

Starting XGB prediction for strategy_variant 58
0 records deleted for strategy_variant 58
1 records inserted for NIFTY strategy_variant 58
completed in 0 seconds

Starting XGB prediction for strategy_variant 59
0 records deleted for strategy_variant 59
1 records inserted for NIFTY strategy_variant 59
completed in 0 seconds

Starting XGB prediction for strategy_variant 60
0 records deleted for 

0 records deleted for strategy_variant 105
1 records inserted for NIFTY strategy_variant 105
completed in 0 seconds

Starting XGB prediction for strategy_variant 106
0 records deleted for strategy_variant 106
1 records inserted for NIFTY strategy_variant 106
completed in 0 seconds

Starting XGB prediction for strategy_variant 107
0 records deleted for strategy_variant 107
1 records inserted for NIFTY strategy_variant 107
completed in 0 seconds

Starting XGB prediction for strategy_variant 108
0 records deleted for strategy_variant 108
1 records inserted for NIFTY strategy_variant 108
completed in 0 seconds

Starting XGB prediction for strategy_variant 109
0 records deleted for strategy_variant 109
1 records inserted for NIFTY strategy_variant 109
completed in 0 seconds

Starting XGB prediction for strategy_variant 110
0 records deleted for strategy_variant 110
1 records inserted for NIFTY strategy_variant 110
completed in 0 seconds

Starting XGB prediction for strategy_variant 111
0 re

0 records deleted for strategy_variant 156
1 records inserted for NIFTY strategy_variant 156
completed in 0 seconds

Starting XGB prediction for strategy_variant 157
0 records deleted for strategy_variant 157
1 records inserted for NIFTY strategy_variant 157
completed in 0 seconds

Starting XGB prediction for strategy_variant 158
0 records deleted for strategy_variant 158
1 records inserted for NIFTY strategy_variant 158
completed in 0 seconds

Starting XGB prediction for strategy_variant 159
0 records deleted for strategy_variant 159
1 records inserted for NIFTY strategy_variant 159
completed in 0 seconds

Starting XGB prediction for strategy_variant 160
0 records deleted for strategy_variant 160
1 records inserted for NIFTY strategy_variant 160
completed in 0 seconds

Starting XGB prediction for strategy_variant 161
0 records deleted for strategy_variant 161
1 records inserted for NIFTY strategy_variant 161
completed in 0 seconds

Starting XGB prediction for strategy_variant 162
0 re

0 records deleted for strategy_variant 207
1 records inserted for NIFTY strategy_variant 207
completed in 0 seconds

Starting XGB prediction for strategy_variant 208
0 records deleted for strategy_variant 208
1 records inserted for NIFTY strategy_variant 208
completed in 0 seconds

Starting XGB prediction for strategy_variant 209
0 records deleted for strategy_variant 209
1 records inserted for NIFTY strategy_variant 209
completed in 0 seconds

Starting XGB prediction for strategy_variant 210
0 records deleted for strategy_variant 210
1 records inserted for NIFTY strategy_variant 210
completed in 0 seconds

Starting XGB prediction for strategy_variant 211
0 records deleted for strategy_variant 211
1 records inserted for NIFTY strategy_variant 211
completed in 0 seconds

Starting XGB prediction for strategy_variant 212
0 records deleted for strategy_variant 212
1 records inserted for NIFTY strategy_variant 212
completed in 0 seconds

Starting XGB prediction for strategy_variant 213
0 re

0 records deleted for strategy_variant 258
1 records inserted for NIFTY strategy_variant 258
completed in 0 seconds

Starting XGB prediction for strategy_variant 259
0 records deleted for strategy_variant 259
1 records inserted for NIFTY strategy_variant 259
completed in 0 seconds

Starting XGB prediction for strategy_variant 260
0 records deleted for strategy_variant 260
1 records inserted for NIFTY strategy_variant 260
completed in 0 seconds

Starting XGB prediction for strategy_variant 261
0 records deleted for strategy_variant 261
1 records inserted for NIFTY strategy_variant 261
completed in 0 seconds

Starting XGB prediction for strategy_variant 262
0 records deleted for strategy_variant 262
1 records inserted for NIFTY strategy_variant 262
completed in 0 seconds

Starting XGB prediction for strategy_variant 263
0 records deleted for strategy_variant 263
1 records inserted for NIFTY strategy_variant 263
completed in 0 seconds

Starting XGB prediction for strategy_variant 264
0 re

0 records deleted for strategy_variant 309
1 records inserted for NIFTY strategy_variant 309
completed in 0 seconds

Starting XGB prediction for strategy_variant 310
0 records deleted for strategy_variant 310
1 records inserted for NIFTY strategy_variant 310
completed in 0 seconds

Starting XGB prediction for strategy_variant 311
0 records deleted for strategy_variant 311
1 records inserted for NIFTY strategy_variant 311
completed in 0 seconds

Starting XGB prediction for strategy_variant 312
0 records deleted for strategy_variant 312
1 records inserted for NIFTY strategy_variant 312
completed in 0 seconds

Starting XGB prediction for strategy_variant 313
0 records deleted for strategy_variant 313
1 records inserted for NIFTY strategy_variant 313
completed in 0 seconds

Starting XGB prediction for strategy_variant 314
0 records deleted for strategy_variant 314
1 records inserted for NIFTY strategy_variant 314
completed in 0 seconds

Starting XGB prediction for strategy_variant 315
0 re

0 records deleted for strategy_variant 360
1 records inserted for NIFTY strategy_variant 360
completed in 0 seconds

Starting XGB prediction for strategy_variant 361
0 records deleted for strategy_variant 361
1 records inserted for NIFTY strategy_variant 361
completed in 0 seconds

Starting XGB prediction for strategy_variant 362
0 records deleted for strategy_variant 362
1 records inserted for NIFTY strategy_variant 362
completed in 0 seconds

Starting XGB prediction for strategy_variant 363
0 records deleted for strategy_variant 363
1 records inserted for NIFTY strategy_variant 363
completed in 0 seconds

Starting XGB prediction for strategy_variant 364
0 records deleted for strategy_variant 364
1 records inserted for NIFTY strategy_variant 364
completed in 0 seconds

Starting XGB prediction for strategy_variant 365
0 records deleted for strategy_variant 365
1 records inserted for NIFTY strategy_variant 365
completed in 0 seconds

Starting XGB prediction for strategy_variant 366
0 re

In [5]:
start = time.time()
self.Live_Strategies()
print(time.time()-start)

165888  records deleted for NIFTY
166272 records inserted for NIFTY
4.674954891204834
